In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import json
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from tensorflow.keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax,Nadam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import timeit

In [ ]:
# import jsonlines
# with jsonlines.open('output.jsonl', 'r').readline()
#     writer.write_all(items)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
start = timeit.default_timer()
dataframe = pd.read_csv('/content/drive/MyDrive/Research/Deep Learning/LaminatedPlate_RZT_Q9.csv')
dataset = dataframe.values
data = dataset[:,0:55]
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

In [ ]:
start = timeit.default_timer()
nb_set = 50000
nb_test=int(nb_set*0.8)

x_train = data[:nb_test,0:50]
x_test  = data[nb_test:,0:50]

y_train = data[:nb_test,50:55]
y_test  = data[nb_test:,50:55]
print(np.shape(y_train))
print("y_test:",np.shape(y_test))
print('y_train = ',y_train)
print('y_test=',y_test)



(40000, 5)
y_test: (9999, 5)
y_train =  [[0.42717531 0.58751401 0.54305166 0.44421383 0.45133866]
 [0.40574368 0.51058079 0.67147243 0.46764448 0.37195867]
 [0.35747964 0.64572483 0.67587772 0.34620174 0.35082198]
 ...
 [0.2701243  0.73861164 0.76651982 0.23882662 0.25368718]
 [0.4142306  0.62034412 0.66766787 0.37549006 0.35519023]
 [0.4390913  0.52040345 0.59644907 0.46718325 0.4322217 ]]
y_test= [[0.221003   0.72325137 0.82458951 0.26082745 0.20131517]
 [0.37231033 0.59483156 0.65765585 0.41275771 0.35312353]
 [0.52961852 0.48671633 0.42651181 0.52838891 0.55180836]
 ...
 [0.26840977 0.69477223 0.76418369 0.29302154 0.23090653]
 [0.48418345 0.51783242 0.50981177 0.51196901 0.46970409]
 [0.45983712 0.55501351 0.51388333 0.47590056 0.47421325]]


In [ ]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

input_node  = x_train.shape[1]
output_node = y_train.shape[1]
n_timesteps = x_train.shape[2]
print("inputnode",input_node)
print("inputnode",output_node)

inputnode 50
inputnode 5


In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(200, return_sequences=True,input_shape=(input_node,n_timesteps), activation="relu"))
model.add(LSTM(100, return_sequences=True,activation="relu"))
model.add(LSTM(100, return_sequences=True,activation="relu"))
model.add(LSTM(50, activation="relu"))
model.add(Dense(output_node))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 200)           161600    
                                                                 
 lstm_1 (LSTM)               (None, 50, 100)           120400    
                                                                 
 lstm_2 (LSTM)               (None, 50, 100)           80400     
                                                                 
 lstm_3 (LSTM)               (None, 50)                30200     
                                                                 
 dense (Dense)               (None, 5)                 255       
                                                                 
Total params: 392,855
Trainable params: 392,855
Non-trainable params: 0
_________________________________________________________________


In [ ]:
adam = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
rms = RMSprop(learning_rate = 0.01, rho=0.9, epsilon=1e-08)
ada = Adagrad(learning_rate=0.2, epsilon=1e-08)
sgd = SGD(learning_rate=0.1, momentum=0.1)
adadel=Adadelta(learning_rate=0.1, rho=0.95, epsilon=1e-08)
adamax = Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
nadam = Nadam(learning_rate=0.002, beta_1=0.9, beta_2=0.999)

In [ ]:
import tensorflow as tf
model.compile(loss='mse', optimizer=adamax,metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=1000, batch_size=256, verbose=1) 
scores = model.evaluate(x_test, y_test, verbose=1)
model.save_weights('target_weight.h5', overwrite=True)

In [ ]:

pred_train = model.predict(x_train)
pred_test = model.predict(x_test)
mse_train = mean_squared_error(y_train,pred_train)
mse_test  = mean_squared_error(y_test,pred_test)
mae_train = mean_absolute_error(y_train,pred_train)
mae_test = mean_absolute_error(y_test,pred_test)
r2_train = r2_score(y_train,pred_train)
r2_test = r2_score(y_test,pred_test)

In [ ]:
stop = timeit.default_timer()
with open("MSE_Computingtime.txt", "w") as text_file:
    print("Computing time: "+str(stop - start), file=text_file)
    print('MSE for train: ' +str(mse_train),file=text_file)
    print('MSE for test: '  + str(mse_test), file=text_file)
    print('mean absolute error for train: '+str(mae_train),file=text_file)
    print('mean absolute error for test: ' + str(mae_test), file=text_file)
    print('r2 score for train:' +str(r2_train),file=text_file)
    print('r2 score for test: ' +str(r2_test),file=text_file)
    print('scores: ' +str(scores),file=text_file)
    print("%s:%.2f%%" % (model.metrics_names[1], scores[1] * 100),file=text_file)


print('Computing time:',stop - start)
print('mean square error for train and test',mse_train, mse_test)
print('mean absoliute error for train and test', mae_train,mae_test)
print('r2 score for train and test', r2_train,r2_test)
print('scores ',scores)
print("%s:%.2f%%"%(model.metrics_names[1],scores[1]*100))


In [ ]:
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['accuracy'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper right')
# plt.show()

In [ ]:
# model.summary()
# config = model.get_config()
# model.get_weights()
# print('config',config)
# print(ascii(model))